In [ ]:
%run common_init.py

# Direct detection of Dark matter using different target materials #

Author:

Joran Angevaare <j.angevaare@nikef.nl>

Date:

14 october 2019 

## Goal ## 

- Roughly reproduce <https://journals.aps.org/prd/abstract/10.1103/PhysRevD.83.083505>
- Update the results thereof with more recent knowledge of the DM-distribution

### Approach ###
To achieve these goals, we must first get a decent recoil spectrum, that is flexible enough to have different astrophysical parameters. Further, it must also be flexible enough to be able to allow for different analytic functions to be added to the model. For instance, we must be able to incorporate the $\rm{SHM}^{++}$ as presented here <https://arxiv.org/abs/1810.11468>.

When we have a sufficiently flexible model, we want to add in the detector physics, we should therein incorporate at least the following parameters:
- target
  - material
  - cross-section
- threshold
- background
- cut efficiency  
- volume
- exposure

Finally we should perform the inference

In [ ]:
from datetime import datetime

In [ ]:
from pymultinest.solve import solve as multines_solve

In [ ]:
stats = dddm.NestedSamplerStatModel('Xe')

In [ ]:
stats._log_probability_nested([0,0])

In [ ]:
time.time()

In [ ]:
from pymultinest.solve import solve

In [ ]:
from __future__ import absolute_import, unicode_literals, print_function
import numpy
from numpy import pi, cos

In [ ]:
# %time
# start_multi = time.time()

# result = solve(LogLikelihood=stats._log_probability_nestle, 
#       Prior=stats._log_prior_transform_nestle,
#       n_dims = 2,
#       outputfiles_basename = 'temp_res2',
#       verbose=True,
#      )
# end_multi = time.time()

In [ ]:
stats2 = dddm.NestedSamplerStatModel('Xe')
stats2.sampler = 'multinest'
stats2.nlive = 10
stats2.verbose = 0
stats2.config['save_intermediate'] = True

In [ ]:
stats2.save_results()

In [ ]:
for a in stats2.result.keys():
    print(a + 'a')

In [ ]:
fit_summary = stats2.get_summary()
fit_summary

In [ ]:
for x in stats2.result.keys():
    print((x))

In [ ]:
resdict = {}
for name, col in zip(stats2.fit_parameters, stats2.result['samples'].transpose()):
                print('%15s : %.3f +- %.3f' % (name, col.mean(), col.std()))
                # TODO
                #  add weights?
                resdict[name + "_fit_res"] = ("{0:5.2f} +/- {1:5.2f}".format(col.mean(),col.std()))
                if "log_" in name:
                    resdict[name[4:] + "_fit_res"] = "%.3g +/- %.2g" % (10 ** col.mean(), 10 ** (col.mean()) * np.log(10) * col.std())
                    print('\t', name[4:], resdict[name[4:] + "_fit_res"])

In [ ]:
resdict

In [ ]:
start_nestle = time.time()
stats2.run_multinest()
end_nestle = time.time()

In [ ]:
stats2.result

In [ ]:
end_nestle - start_nestle

In [ ]:
stats2.get_summary()

In [ ]:
%run common_init.py

In [ ]:
stats3 = dddm.NestedSamplerStatModel('Xe')

stats3.nlive = 200
stats3.verbose = 1
stats3.sampler = 'multinest'

In [ ]:
start_nestle = time.time()
stats3.run_multinest()
end_nestle = time.time()

In [ ]:
end_nestle - start_nestle

In [ ]:
stats3.log


In [ ]:
!ls /mnt/c/Users/Joran/dddm_data/results/nested_multinest2/multinest0

In [ ]:
from pymultinest import Analyzer

In [ ]:
# create analyzer object
a = Analyzer(2, outputfiles_basename = "/mnt/c/Users/Joran/dddm_data/results/nested_multinest2/multinest0/multinest")

# get a dictionary containing information about
#   the logZ and its errors
#   the individual modes and their parameters
#   quantiles of the parameter posteriors
stats = a.get_stats()

# get the best fit (highest likelihood) point
bestfit_params = a.get_best_fit()

# iterate through the "posterior chain"
for i, params in enumerate(a.get_equal_weighted_posterior()):
        print (i, params)

In [ ]:
a.get_mode_stats()

In [ ]:
bestfit_params

In [ ]:
np.shape(a.get_data())

In [ ]:
plt.hist(a.get_data()[:,0])
plt.yscale("log")

In [ ]:
plt.hist(a.get_data()[:,1])
plt.yscale("log")

In [ ]:
plt.hist(a.get_data()[:,2])
plt.yscale("log")

In [ ]:
plt.hist(a.get_data()[:,3])
plt.yscale("log")

In [ ]:
plt.hist(stats2.result['samples'][:,1])

In [ ]:
stats2.save_results()

In [ ]:
stats2.log['saved_in']

## Distribution of the DM ##
First we need to make a DM-rate spectrum

In [ ]:
stats_full = dddm.NestedSamplerStatModel('Xe_migd')

stats_full.nlive = 200
stats_full.verbose = 1
stats_full.sampler = 'multinest'

In [ ]:
stats_full.log
stats_full.set_prior('migdal_lower')

In [ ]:
# for key in stats_full.log.keys():
#     stats_full.log[key] = False


In [ ]:
stats_full.fit_parameters = stats_full.known_parameters

In [ ]:
stats_full.check_spectrum()

In [ ]:
start = time.time()
stats_full.run_multinest()
end = time.time()

In [ ]:
end-start

In [ ]:
stats_full.save_results()

In [ ]:
9170.64813709259/3600

In [ ]:
assert stats_full.log['did_run']

In [ ]:
stats_full.show_walkers()

In [ ]:
stats_full.show_corner()

In [ ]:
corner.corner(
    stats_full.sampler.get_chain(
        flat=True,
        thin = 50,
        discard=int(stats_full.nsteps * 0.2)
    ),
              labels=stats_full.fit_parameters);